# SC42x 
## 자연어처리 (Natural Language Processing)

# Part 1 : 개념 요약

> 다음의 키워드(총 9개)에 대해서 **한 줄**로 간단하게 요약해주세요.

**N421**
- Stop words(불용어) : I, my, me, over, 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는 데는 거의 기여하는 바가 없는 단어들을 의미한다.  
- 통계적 트리밍 : 기존에 알려진 불용어를 제거하는 대신 코퍼스에서 통계적인 방법을 통해 단어를 제거하는 방법
- Stemming과 Lemmatization
  * stemming은 어간추출로써 어형이 변형된 단어로부터 접사 등을 제거하고 그 단어의 어간을 분리해 내는 것을 의미한다.  
  * Lemmatization는 단어들이 다른 형태를 가지고 있더라도 그 뿌리 단어를 찾아가서 단어의 개수를 줄일 수 있는지를 판단하는 것입니다.(즉, 형태학적 파싱)

**N422**
- CountVectorizer : 단어들의 출현 빈도로 여러개의 문서를 벡터화하는 함수
- TfidfVectorizer :여러 개의 문서가 있을 때, 각각의 문서의 내에 있는 단어들에 수치값을 주는 방법인데, 가중치가 적용되어있다. TF-IDF를 계산하면 문서 내에 상대적으로 중요한 단어를 알 수 있다.
- LSA : 데이터의 차원을 축소해 문서들에 숨어있는(latent) 의미(Topics)를 끌어내는 방법

**N423**
- word2vec : 백터화된 문장의 문맥 정보를 보존하기 백터들의 분산을 이용하여 유사도 예측하는 분석방법입니다. 그중 CBOW와 skip-gram이라는 2가지 방법을 학습하였으며, CBOW는 문맥단어들을 가지고 타겟 단어 하나를 맞추는 방법이며, skip-gram은 cbow와 반대되는 개념입니다. (skip-gram : 중간에 있는 단어로 주변 단어를 예측함)

- RNN : RNN은 기존의 신경망에서는 입력 순서를 고려한 학습 모델입니다. RNN이 파생된 이유는 시계열 데이터를 적절하게 표현할 수 없기 때문입니다.

- LSTM, GRU
 * LSTM :  RNN에 Gate를 추가한 모델
 * GRU : 게이트 메커니즘이 적용된 rnn 프레임워크 일종으로 LSTM보다 더 간략한 구조를 가지고 있는 모델

# Part 2 : Yelp 데이터 다루기

한 주간 자연어처리 기법을 배우면서 여러분은 다양한 기술들을 접했습니다. 어떻게 텍스트 데이터를 다뤄야 하는지, 텍스트를 벡터화 하는 법, 문서에서 토픽을 모델하는 법 등 다양한 NLP 기법을 배웠는데요. 이번 스프린트 챌린지에선 가장 유명한 NLP 데이터셋 중 하나인 [Yelp](https://www.yelp.com/dataset)의 리뷰 데이터셋을 통해 배운 것들을 복습해보는 시간을 갖겠습니다. 

실제 데이터셋은 매우 방대하기 때문에 이번 스프린트 챌린지에선 샘플링을 통해 일부의 데이터만 다루도록 하겠습니다. 관심이 있는 분들은 스프린트 챌린지 이후에 다뤄보시기 바랍니다. (미리 분석하고 싶거나 알아보고 싶은 부분을 노트로 남겨두면 이후에 도움이 될 것 같습니다)

In [3]:
import re
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import spacy

In [4]:
import pandas as pd

# yelp 데이터셋을 불러옵니다.
yelp = pd.read_json('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/yelp/yelp_reviews_sc41x.json')
print(yelp.shape)
yelp.head()

(50000, 9)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


## 2.1 `tokenize` 함수를 완성하세요. 

In [5]:
# 줄바꿈제거코드 
yelp['text_preprocessed'] = yelp['text'].apply(lambda x: x.replace('\n',''))

# 정규화을 사용한 대문자->소문자치환 및  특수문자 제거
yelp['text_preprocessed'] = yelp['text'].apply(lambda x:re.sub(r"[^a-zA-Z0-9 ]", "", x)).str.lower()
yelp.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,text_preprocessed
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16,as someone who has worked with many museums i ...
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52,i am actually horrified this place is still in...
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11,i love deagans i do i really do the atmosphere...
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52,dismal lukewarm defrostedtasting texmex glopmu...
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57,oh happy day finally have a canes near my casa...


In [6]:
# en_core_web_sm load
nlp = spacy.load("en_core_web_sm")

In [7]:
# spacy tokenizer 함수 zhem 
def tokenize(document):
    doc = nlp(document)
    # punctuations: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True) and (token.is_alpha == True)]

## 2.2 : 리뷰들을 벡터로 표현하세요

In [8]:
# CountVectorizer를 이용한 백터화 
vect = CountVectorizer(stop_words='english'
                        ,tokenizer=tokenize
                        ,max_features = 4000
                       )

dtm = vect.fit_transform(yelp['text_preprocessed'])
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
dtm.head()

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['nt'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,aaa,ability,able,absolute,absolutely,ac,acai,accent,accept,acceptable,...,york,young,yr,yuck,yum,yummy,zero,zombie,zone,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.3 가짜 리뷰를 작성한 후에 비슷한 리뷰를 10개 출력해보세요. 어떤 패턴을 발견하셨나요?
> 데이터셋의 사이즈로 인해 `NearestNeighbors` 모델을 사용하는 것을 권장합니다.

In [9]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

# Test
example_st = "it was good"

test_st = vect.transform([example_st])

# print full text 
pd.options.display.max_colwidth = 100


# result
print("Test 문장 : " ,example_st)
print("------------------------------------")
for index_nm, row in enumerate(nn.kneighbors(test_st.todense())[1][0]):
    print("{0}번째 문장 : \n {1}".format(index_nm+1, yelp['text'][row]))
    print('\n')

Test 문장 :  it was good
------------------------------------
1번째로 문장 : 
 -good location
-good prices
-clean rooms
-affordable food
-friendly and helpful staff
-NO BEDBUGS!!!


2번째로 문장 : 
 :)


3번째로 문장 : 
 去吃了烧烤了，鱼豆腐好好吃，外酥里嫩！肉也很好吃，一进去服务员都很热情！有种宾至如归的感觉！环境也很好！我还吃了绝代双骄，就是我想要的那个味道！特别推荐大家去试试，外婆家红烧肉，也就是我记忆中的味道！很正宗！以后会常来的！希望可以继续保持哦！


4번째로 문장 : 
 如果做创新餐厅要丢掉传统菜的口味和优点，那还是不要创新了。天妇罗互相之间粘在一起，鳗鱼卷两个两个粘着而且米饭是松松的。不想说了。扫兴


5번째로 문장 : 
 Good food.


6번째로 문장 : 
 タイ料理屋さん。
夕食分と朝食分を持ち帰りで注文。自分、日本人なんだけど、辛さはどのくらいが良さそう？って聞いて、ミディアムにしたんだけど、ミディアムでもだいぶ辛かった。。。もいちょい下のランクにしても良いかも。
でも、麺もカレーもどちらも美味しかったので、また行きたいお店。


7번째로 문장 : 
 A good experience


8번째로 문장 : 
 入店時に精算用のカードを手渡され、好きな席に座り、キッチンやバーに好きなものを買いに行きます。
ワインが豊富。
パスタは麺の種類がえらべ、ピザはナポリっぽい感じで美味しいです。
サラダの味もなかなかでした。
カジュアルで、でも美味しく気軽に入れるお店です。
帰りにカードを精算します。
なのでチップも不要。
お一人でもどうぞ。


9번째로 문장 : 
 这个地方很棒，我喜欢他们的玉米棒！


10번째로 문장 : 
 周末请饮茶，听讲唔错，琳住吃好d就选择离呢度。12点几仍然要等位，假期无办法啦。门口入去大厅会见到隐蔽鱼缸养住鲜活大皇帝蟹和大龙虾。餐馆大厅几大气，高大上，特别是寿司吧的位置。可能晚左离，想试招牌点心黑金流沙包卖晒了，点了黑金奶油糕，其实就系黑芝麻糕加奶油，因为都几大一件，好饱肚，少少痴口。翡翠杂菌饺和樱花虾墨鱼卖

## 2.4 리뷰의 별 갯수 ('stars')를 예측하는 분류 Pipeline을 구축하세요.
> `CountVectorizer` 혹은 `TfidfVector`가 포함 되있어야 합니다. 

In [10]:
# 기본 라이브러리 
import numpy as np
import seaborn as sns

# 분석 라이브러리 
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD

# 구축 및 평가 라이브러리 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [11]:
# rf_model 구현
from sklearn.decomposition import TruncatedSVD

vect = CountVectorizer(stop_words='english',
                      ngram_range=(1,2),
                      min_df=2,
                      max_df=0.6,
                      max_features=5000)

svd_model = TruncatedSVD(algorithm='randomized',
                  n_iter=5,
                  random_state=2)

rf_model =RandomForestClassifier(n_estimators=500, random_state=2)

In [12]:
params = {'svd__n_components' : stats.randint(2, 3)}

In [13]:
pipe = Pipeline([
    ('vect', vect)
    , ('svd', svd_model)
    , ('clf', rf_model)
])

In [14]:
target = yelp['stars']
train = yelp['text_preprocessed']
print(train.shape, target.shape)

(50000,) (50000,)


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=.2, random_state=2)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(40000,) (10000,)
(40000,) (10000,)


In [16]:
random_search_model = RandomizedSearchCV(pipe, params, cv=3, n_iter=5, n_jobs=-1, verbose=1)
random_search_model.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.2min finished


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('vect',
                                              CountVectorizer(max_df=0.6,
                                                              max_features=5000,
                                                              min_df=2,
                                                              ngram_range=(1,
                                                                           2),
                                                              stop_words='english')),
                                             ('svd',
                                              TruncatedSVD(random_state=2)),
                                             ('clf',
                                              RandomForestClassifier(n_estimators=500,
                                                                     random_state=2))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'svd__n_comp

In [17]:
y_pred = random_search_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.3937

## 2.5 구축한 Pipeline으로 2.3에서 작성한 가짜 리뷰의 별 갯수를 예측하세요 

In [22]:
print(random_search_model.predict([example_st]))

[5]


## 2.6 `GridSearchCV` 혹은 `RandomizedSearchCV`를 활용하여 Pipeline을 튜닝해서 성능을 개선하세요.

위에서 한번에 진행함 
아래 램던서치문제가 있는지 사전에 읽지 못하여 한번에 구현을 진행함

# Part 3 : Deep Learning for Sequence

>LSTMs로 무엇을 할 수 있을까요? **시퀀스**를 분석하고 있기 때문에, 우리는 분류 이상의 것을 할 수 있습니다. <br>
>Kaggle에서 Text Summarization 문제를 해결한 노트북을 참고하여 우리가 배운 내용들이 실제로 어떻게 사용되는지 확인해보고 **모델 생성 (아키텍쳐 파트)과 관련된 내용**에 한글로 주석을 달아봅시다.

[Data Link](https://www.kaggle.com/sunnysai12345/news-summary) <br/>
[Notebook Example Link](https://www.kaggle.com/sandeepbhogaraju/text-summarization-with-seq2seq-model)


1. 전처리를 진행한 상태 
2. 인코더 구현 
3. 성능향상을 위해 lstm을 3개 쌓아서 모델을 구성함 
4. 임베딩 layer 생성 
5. 디코더 구현
6. 각 스텝마다 cost (오류)를 계산해서 하위 스텝으로 오류를 전파하여 각 weight를 업데이트하기 위해 TimeDistributed로 dense생성
7. rmsprop으로 최적화를 하고 다중분류이기 때문에 sparse_categorical_crossentropy로 loss함수 정의함
8. 튀는 현상 및 학습속도향상을 위해 EarlyStopping을 사용함
9. 모델 예측을 위한 초기화 
10. 목표 어휘 이상의 prob dist를 생성을 위해 고밀도 소프트 맥스 레이어 정의
11. 요약 및 리뷰를 위해 정수 시퀀스를 단어 시퀀스로 변환하는 함수정의
12. 모델실행 

# Advanced Goals: 3점을 획득하기 위해선 아래의 조건 중 하나 이상을 만족해야합니다
 
- Part 2에서 vectorization 방법들을 모두 사용한 후 비교해보세요
- SC에서 사용한 데이터보다 더 큰 Yelp 데이터셋을 분석해보세요 (데이터가 매우 크기 때문에 파일을 읽어올 때 주의하시길 바랍니다: $\approx$ 6.6+ GB)
- Part 3에서 사용한 노트북 전체에 주석을 달아봅니다.

In [23]:
tfid = TfidfVectorizer(stop_words='english',
                      ngram_range=(1,2),
                      min_df=2,
                      max_df=0.6,
                      max_features=5000)

svd_model = TruncatedSVD(algorithm='randomized',
                  n_iter=5,
                  random_state=2)

rf_model =RandomForestClassifier(n_estimators=500, random_state=2)

params = {'svd__n_components' : stats.randint(2, 3)}

pipe = Pipeline([
    ('tfid', tfid)
    , ('svd', svd_model)
    , ('clf', rf_model)
])

random_search_model = RandomizedSearchCV(pipe, params, cv=3, n_iter=5, n_jobs=-1, verbose=1)
random_search_model.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.1min finished


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('tfid',
                                              TfidfVectorizer(max_df=0.6,
                                                              max_features=5000,
                                                              min_df=2,
                                                              ngram_range=(1,
                                                                           2),
                                                              stop_words='english')),
                                             ('svd',
                                              TruncatedSVD(random_state=2)),
                                             ('clf',
                                              RandomForestClassifier(n_estimators=500,
                                                                     random_state=2))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'svd__n_comp

In [24]:
y_pred = random_search_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.4063

In [26]:
random_search_model.predict([example_st]).tolist()

[5]